In [1]:
%load_ext autoreload
# %reload_ext autoreload

# Reload all modules imported with %aimport every time before executing the Python code typed
%autoreload 1

%aimport context_nn 
%aimport phrase_feeder 
%aimport bitnotes
%aimport watch_point
%aimport cluster
%aimport constants
import numpy as np
from bitarray import bitarray
from context_nn import ContextNN
from watch_point import WatchPoint
from cluster import Cluster
from phrase_feeder import PhraseFeeder
from bitnotes import BitNotes
from pprint import pprint
import math
import constants as const

In [2]:
import pickle

def load_cxnn(file_name: str) -> ContextNN:
    with open(file_name, 'rb') as f:
        cxnn = pickle.load(f)
        return cxnn
    
def load_noted_phrases(file_name: str) -> list:
    with open(file_name, 'rb') as f:
        noted_phrases = pickle.load(f)
        return noted_phrases

In [3]:
cxnn = load_cxnn('./data/cxnn_langs.pickle')
noted_phrases = load_noted_phrases('./data/texts/tst_rus.pickle')

In [36]:
cxnn = load_cxnn('./data/cxnn_monotone.pickle')
noted_phrases = load_noted_phrases('./data/texts/tst_bbb.pickle')

In [ ]:
cxnn = load_cxnn('./data/cxnn_probs.pickle')
noted_phrases = load_noted_phrases('./data/texts/tst_3331.pickle')

In [44]:
cxnn = load_cxnn('./data/cxnn_dna.pickle')
noted_phrases = load_noted_phrases('./data/texts/tst_cdef.pickle')

In [4]:
%%time

cxnn.start_detection()

for num, phrase in enumerate(noted_phrases[:2500]):
    bit_chord = cxnn.bit_notes.phrase_chord_as_bits(phrase)
    cxnn.detect_bits(bit_chord)
    if num % 500 == 0:
        print(f'phrase # {num}/{len(noted_phrases)}')

phrase # 0/2755
phrase # 500/2755
phrase # 1000/2755
phrase # 1500/2755
phrase # 2000/2755
Wall time: 2min 36s


In [7]:
print(cxnn.cluster_count())
pprint(cxnn.cluster_len_stats())
pprint(cxnn.cluster_activity_stats())

57766
{5: 1953, 6: 13446, 7: 33179, 8: 7589, 9: 1395, 10: 176, 11: 25, 12: 3}
{0: 57766}


In [5]:
cxnn.data_marks

{(0,): 'bel',
 (1,): 'blg',
 (2,): 'eng',
 (3,): 'epo',
 (4,): 'jbo',
 (5,): 'pol',
 (6,): 'rus',
 (7,): 'ukr'}

In [6]:
bit_detections = {}
for point in cxnn.point_objects:
    key = point.output_bit
    bit_detections[key] = bit_detections.get(key, 0) + point.detections
    
print(sorted(bit_detections.items()))
print(sorted(bit_detections.items(), key=lambda x: x[1], reverse=True))

[(0, 754685), (1, 521326), (2, 2070794), (3, 1542230), (4, 1064950), (5, 1266822), (6, 223919), (7, 570302)]
[(2, 2070794), (3, 1542230), (5, 1266822), (4, 1064950), (0, 754685), (7, 570302), (1, 521326), (6, 223919)]


In [9]:
detection = cxnn.summarize_detection()

0 0.0
1 0.0
2 0
3 0.0
4 0.0
5 0
6 0.0
7 0.0


In [7]:
for bits, label in cxnn.data_marks.items():
    rate = np.sum(detection[list(bits)])
    print(label, rate)

NameError: name 'detection' is not defined

In [8]:
cxnn.vectors_received

2500

In [9]:
print(cxnn.cluster_count())
points = [(bit, count) for bit, count in cxnn.point_stats() if count > 0]
# print(sorted(points, key=lambda x: x[0]))
bit_points = {}
for bit, count in points:
    bit_points[bit] = bit_points.get(bit, 0) + 1

print(sorted(bit_points.items()))
    
bit_clusters = {}
for bit, count in points:
    bit_clusters[bit] = bit_clusters.get(bit, 0) + count

print(sorted(bit_clusters.items()))

bit_activations = {}
for point in cxnn.point_objects:
    key = point.output_bit
    bit_activations[key] = bit_activations.get(key, 0) + sum(cluster.activation_count() 
                                                             for cluster in point.cluster_objects 
                                                             if cluster.consolidations >= 0 and 
                                                                cluster.activation_count() > 200)

print(sorted(bit_activations.items()))

bit_divs = {}
for key in bit_clusters.keys():
    bit_divs[key] = bit_activations[key] / (bit_clusters[key] * bit_points[key])

pprint(sorted(bit_divs.items()))


57766
[(0, 226), (1, 221), (2, 246), (3, 249), (4, 245), (5, 241), (6, 195), (7, 225)]
[(0, 4448), (1, 2649), (2, 11712), (3, 11401), (4, 8478), (5, 14890), (6, 1194), (7, 2994)]
[(0, 0), (1, 0), (2, 0), (3, 0), (4, 0), (5, 0), (6, 0), (7, 0)]
[(0, 0.0), (1, 0.0), (2, 0.0), (3, 0.0), (4, 0.0), (5, 0.0), (6, 0.0), (7, 0.0)]


In [53]:
212356 / 933

227.60557341907824

In [55]:
3031768 / 10411

291.2081452310057